In [1]:
import re
import nltk
import numpy as np
import pandas as pd
import pickle
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV

In [2]:
fileLocation = "data.csv"
target = 'rating'
col_to_transform = ['title', 'text']

In [3]:
data = pd.read_csv(fileLocation, encoding='cp1251', sep=',', )
stemmer = SnowballStemmer("russian")

# убираем неиспользуемые колонки

In [4]:
data.drop(['Unnamed: 0', 'date', 'bank'], axis=1, inplace=True)
data.dropna(axis=0, inplace=True)

# унифицируем данные

In [5]:
f = lambda x: re.sub("[,.!:(){}/\-_']+", ' ', x)
stem_wrds = lambda x: [stemmer.stem(s) for s in x if x not in stopwords.words('russian')]
binarize_target = lambda x: 1 if x/5.0 > 0.5 else 0 

In [6]:
for col in col_to_transform:
    data[col] = data[col].map(f).str.lower()
    data[col] = data[col].str[0:250]
#    data[col] = data[col].str.split()
#    data[col] = data[col].map(stem_wrds)

In [7]:
data[target] = data[target].map(binarize_target)

объединим заколовок и сам текст отзыва

In [8]:
data['all_text'] = data.title + data.text

In [9]:
data.drop(col_to_transform, axis = 1, inplace = True)

# построим bag of words

In [10]:
vectorizer = CountVectorizer(stop_words = stopwords.words('russian'))
X = vectorizer.fit(data.all_text)
X = vectorizer.transform()

In [11]:
feature_names = np.array(vectorizer.get_feature_names())
feature_names[X[0].nonzero()[1]]

array(['вкладе', 'договора', 'заключении', 'указывала', 'который',
       'телефон', 'сотовый', 'звонок', 'сделан', 'времени', 'челябинского',
       'местного', 'телефона', 'услуг', 'своих', 'навязывания', 'данные',
       'личные', 'используют', 'вклада', 'окончания'],
      dtype='<U31')

# построим линейный классификатор

In [16]:
#params = {'penalty': ['l1', 'l2'], 'C': [5, 10, 100, 200, 500, 1000, 10000], 'solver': ['lbfgs', 'liblinear', 'saga']}  
model = LogisticRegression(C=100)

In [17]:
#clf = GridSearchCV(model, params, error_score=0.0)
model.fit(X, data[target])
res = cross_val_score(model, X, data[target], scoring="accuracy", cv=5)
print (res.mean())

0.873353820991


# сохраним модель в pickle

In [44]:
filename_1 = 'model.sav'
filename_2 = 'vectorizer.sav'

In [45]:
pickle.dump(model, open(filename_1, 'wb'))
pickle.dump(vectorizer, open(filename_2, 'wb'))

# проверка на тестовых данных

In [ ]:
test_data = 'отличный банк'

In [51]:
model = pickle.load(open(filename_1, 'rb'))
vectorizer = pickle.load(open(filename_2, 'rb'))
result = model.predict(vectorizer.transform([test_data]))
print (result)

[1]


In [55]:
data

,rating,all_text
0,0,после окончания вклада используют личные данны...
1,1,отличный банк пользуюсь по рекомендации друга ...
2,0,бардак в тинкофф банкеобслуживаюсь в банке дос...
3,0,навязывают кредитные карты обманпримерно в г...
4,0,закрытие вклада без заявления сегодня заканчив...
6,0,штраф за не продление полиса каско года я шиш...
7,1,всё путём давно собирался написать отзыв поль...
10,0,банк не для дистанционного обслуживаниярешил о...
11,0,премиальное обслуживаниедобрый день в москве в...
12,0,ежедненые звонки из банкаежедневно на мой номе...
